## Python Programming - Homework 3

**_Name: Aditya Ramesh Parab_**

### Question 1 
An example file called stock.txt is included in the homework files. The file contains daily headline of Apple inc.’s stock price and movement. You are required to write a program that reads the file, extract the date and stock price from each day’s headline, and list them below. 

Algorithm:
1. Open the stock.txt file
2. Read entire file
3. Regex to extract Date which is in the format Month Day
<br>_(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+(\d{1,2})_
<br> The Month will extract month either be it just first 3 letters or whole month name followed by 1 or 2 digits with space between Month and Date
4. Regex to extract stock price
<br> _\d+[.]\d+_ decimal number will be extracted
5. Map Dates with Stock prices
6. Print Data

In [1]:
import re

try:
    fhand = open('stock.txt')
    data = fhand.read()
    fhand.close()

    print(data, '\n')
    dates = re.findall('(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+(\d{1,2})',data)
    stock_value = re.findall('\d+[.]\d+',data)
    stock_value = ['$'+ stock for stock in stock_value]


    for index, element in enumerate(dates):
        if len(element[1]) == 1:
            date = '0'+ element[1]
            dates[index] = element[0],date

    print('Apple inc. stock prices:')
    for date, price in zip(dates, stock_value):
        print(date[0],date[1], price)
except:
    print('Something went wrong!')

Rumor of Apple car coming, Apr 28, Apple stock trended higher to 112.05
Market opening Apr 29, Apple rises to $114.86 following earning beat
Apr 30, Apple crashes to 98.29 as tech stocks tumble on fear of rate hike
Analysts say Apple a great buy at $107.45, following yesterday's correction, May 01
Apple opens at 105.60 and trades higher midday May 2
May 3, Apple stock is at a all time high at 116.80 after better than expected iPhone oversea market reporting over 80 million units shipped
Apple stock closes more than 3 percent higher on $121.04 on May 4 

Apple inc. stock prices:
Apr 28 $112.05
Apr 29 $114.86
Apr 30 $98.29
May 01 $107.45
May 02 $105.60
May 03 $116.80
May 04 $121.04


### Question 2
An example file called movie_reviews.txt is included in the homework files.
The file contains names of reviewers and their reviews of different movies in the comma 
separated format. The first string is the name of the reviewer followed by a name of a movie 
and its rating. You are required to write a Python program that computes a similarity score 
between  any  two  reviewers  using  Euclidean  distance

Algorithm:

Program is split in 3 parts.
1. Creating dictionary of the given reviewers data -> _create_dictionary()_ function
2. Precalculating the similarity score of all the users with each other and storing it in a pandas dataframe -> _createDataFrame()_ function
3. Menu driven program for user inputs -> _main()_ function

In [2]:
import os.path
import sys

def create_dictionary(file_name):
    file_exists = os.path.exists(file_name)
    if file_exists:
        try:
            fhand = open(file_name)
            data = fhand.read().rstrip().splitlines()
            fhand.close()

            movie_reviews = {}
            for line in data:
                record = line.split(',')
                dic = {}
                for index in range(1, len(record), 2):
                    dic[record[index]] = float(record[index+1])
                name = record[0].upper()
                movie_reviews[name] = dic

            return movie_reviews
        except:
            print('Something went wrong!')
            sys.exit(0)
    else:
        print('*** File with name', file_name,'not Found')
        sys.exit(0)

In [3]:
import math
import pandas as pd

def createDataFrame(movie_reviews):
    reviewers = list(movie_reviews.keys())
    #print(reviewers)
    pdData=[]

    for i in range(len(reviewers)):
        tup = []
        for j in range(len(reviewers)):
            if reviewers[i] == reviewers[j]:
                tup.append(0)
                continue
            
            dic1 = movie_reviews[reviewers[i]]
            dic2 = movie_reviews[reviewers[j]]
            
            sum = 0
            for title in set(dic1).intersection(dic2):
                sum = sum + pow((dic1[title] - dic2[title]),2)

            euDis = round(math.sqrt(sum), 4)
            tup.append(euDis)
            #print(reviewers[i],'-', reviewers[j], ':', euDis)
        pdData.append(tup)
    df = pd.DataFrame(pdData, columns =reviewers)
    df.index = reviewers
    return df;

In [4]:
def test_input(inp):
    valid = ['1','2','3']
    while(True):
        if inp in valid:
            return inp
        else:
            print('Not a valid input. Please enter either 1, 2 or 3 only')
            inp = input('Enter again : ')

In [5]:
import pandas as pd

def main():
    file_name = input('Give the name of the movie reviews file: ')
    
    movie_reviews = create_dictionary(file_name)
    
    #Uncomment for if user input is not required
    #movie_reviews = create_dictionary('movie_reviews.txt')
    df = createDataFrame(movie_reviews)
    reviewers = list(df)
   
    while True:
        print('\nWhat do you want to do?\nInput 1 for similarity between two reviewers.\nInput 2 for similarity between one reviewer and all others in the database.\nInput 3 to Quit.')
        choice = input('Input your choice: ')
        choice = test_input(choice)
        if choice == '1':
            name1 = input('Provide Reviewer1 name: ')
            name2 = input('Provide Reviewer2 name: ')
            name1 = name1.upper()
            name2 = name2.upper()
            
            if set([name1, name2]).issubset(set(name for name in reviewers)):
                result = df.loc[name1,name2]
                print('\nThe similarity score between',name1,'and',name2,'is: ',result)
            else:
                print('*** Reviewer names (either or all entered)', name1,'and', name2, ', not found in the database ***')
                

        elif choice == '2':
            name = input('Provide Reviewer name: ')
            name = name.upper()
            if name in reviewers:
                result = df.loc[name].values.flatten().tolist()
                print('\nThe Similarity Scores are:')
                for dfName, score in zip(reviewers, result):
                    if name == dfName:
                        continue
                    print(name, '-',dfName, ':', score)
            else:
                print('*** Reviewer name', name,', is not in the database ***')
            
        else:
            print('Goodbye!')
            break;
    

In [6]:
main()

Give the name of the movie reviews file: movie_reviews.txt

What do you want to do?
Input 1 for similarity between two reviewers.
Input 2 for similarity between one reviewer and all others in the database.
Input 3 to Quit.
Input your choice: 2
Provide Reviewer name: Peter

The Similarity Scores are:
PETER - NANCY POLLOCK : 1.8708
PETER - JACK HOLMES : 2.8723
PETER - MARY DOYLE : 1.5811
PETER - DOUG REDPATH : 1.8028
PETER - JILL BROWN : 1.5
PETER - TREVOR CHAPPELL : 2.7386

What do you want to do?
Input 1 for similarity between two reviewers.
Input 2 for similarity between one reviewer and all others in the database.
Input 3 to Quit.
Input your choice: 1
Provide Reviewer1 name: nancy pollock
Provide Reviewer2 name: Trevor Chappell

The similarity score between NANCY POLLOCK and TREVOR CHAPPELL is:  1.9365

What do you want to do?
Input 1 for similarity between two reviewers.
Input 2 for similarity between one reviewer and all others in the database.
Input 3 to Quit.
Input your choice: 3